In [ ]:
!pip install tensorflow tensorflow-hub


   ---------------------------------------- 0.0/375.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/375.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/375.7 MB 2.1 MB/s eta 0:02:59
   ---------------------------------------- 0.8/375.7 MB 1.5 MB/s eta 0:04:17
   ---------------------------------------- 1.0/375.7 MB 1.4 MB/s eta 0:04:21
   ---------------------------------------- 1.6/375.7 MB 1.7 MB/s eta 0:03:43
   ---------------------------------------- 2.1/375.7 MB 2.0 MB/s eta 0:03:11
   ---------------------------------------- 2.4/375.7 MB 1.9 MB/s eta 0:03:18
   ---------------------------------------- 2.6/375.7 MB 1.8 MB/s eta 0:03:26
   ---------------------------------------- 2.9/375.7 MB 1.7 MB/s eta 0:03:36
   ---------------------------------------- 3.4/375.7 MB 1.7 MB/s eta 0:03:37
   ---------------------------------------- 4.2/375.7 MB 2.0 MB/s eta 0:03:11
    --------------------------------------- 5.5/375.7 MB 2.3 MB/s eta 0:02:42


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']

In [ ]:
import cv2
import numpy as np

def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize_with_pad(img, 256, 256)
    input_img = tf.expand_dims(img, axis=0)
    input_img = tf.cast(input_img, dtype=tf.int32)
    return input_img

ModuleNotFoundError: No module named 'cv2'

In [ ]:
!pip install opencv-python

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
   --- ------------------------------------ 3.1/39.5 MB 13.2 MB/s eta 0:00:03
   ------ --------------------------------- 6.8/39.5 MB 15.5 MB/s eta 0:00:03
   ------------ --------------------------- 12.3/39.5 MB 18.4 MB/s eta 0:00:02
   ---------------- ----------------------- 16.0/39.5 MB 18.3 MB/s eta 0:00:02
   -------------------- ------------------- 20.7/39.5 MB 19.0 MB/s eta 0:00:01
   ------------------------- -------------- 25.2/39.5 MB 19.2 MB/s eta 0:00:01
   ----------------------------- ---------- 29.4/39.5 MB 19.2 MB/s eta 0:00:01
   -------------------------------- ------- 32.0/39.5 MB 18.8 MB/s eta 0:00:01
   ----------------------------------- ---- 34.9/39.5 MB 17.9 MB/s eta 0:00:01
   ---------------------------------------  38.8/39.5 MB 18.1 MB/s eta 0:00:01
   ---------------------------------------- 39.5/39.5 MB 17.2 MB/s eta 0

In [ ]:
import cv2
import numpy as np

In [ ]:
def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize_with_pad(img, 256, 256)
    input_img = tf.expand_dims(img, axis=0)
    input_img = tf.cast(input_img, dtype=tf.int32)
    return input_img

In [ ]:
def detect_pose(image_path):
    input_img = load_and_preprocess_image(image_path)
    outputs = movenet(input_img)
    keypoints = outputs['output_0'].numpy()
    return keypoints[0, 0, :, :2]  # [num_keypoints, (y, x)]

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd

# 모델 불러오기
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']

# 이미지 전처리 함수
def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize_with_pad(img, 256, 256)
    input_img = tf.expand_dims(img, axis=0)
    input_img = tf.cast(input_img, dtype=tf.int32)
    return input_img

# keypoint 추출 함수
def detect_pose(image_path):
    input_img = load_and_preprocess_image(image_path)
    outputs = movenet(input_img)
    keypoints = outputs['output_0'].numpy()
    return keypoints[0, 0, :, :2]  # [17, (y, x)]

# 결과 저장 리스트
records = []

# Normal 폴더 경로
normal_root = './frames/Normal'

# normal_01 ~ normal_34 처리 (단, normal_28 제외)
for i in range(1, 35):
    if i == 28:
        continue  # normal_28 제외

    folder_name = f'normal_{i:02d}'
    folder_path = os.path.join(normal_root, folder_name)

    if not os.path.exists(folder_path):
        print(f"[경고] 폴더 없음: {folder_path}")
        continue

    for fname in sorted(os.listdir(folder_path)):
        if fname.endswith('.jpg') or fname.endswith('.png'):
            image_path = os.path.join(folder_path, fname)
            keypoints = detect_pose(image_path)

            for kp_idx, (y, x) in enumerate(keypoints):
                records.append({
                    'video_folder': folder_name,
                    'frame': fname,
                    'keypoint_index': kp_idx,
                    'x': float(x),
                    'y': float(y)
                })

# DataFrame으로 변환 및 저장
df = pd.DataFrame(records)
df.to_csv('keypoints_normal.csv', index=False)
print("✅ Normal keypoints 저장 완료: keypoints_normal1.csv")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
!pip install pandas

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.6 MB 4.2 MB/s eta 0:00:03
   -------- ------------------------------- 2.4/11.6 MB 5.0 MB/s eta 0:00:02
   ------------ --------------------------- 3.7/11.6 MB 5.3 MB/s eta 0:00:02
   -------------- ------------------------- 4.2/11.6 MB 5.3 MB/s eta 0:00:02
   ----------------- ---------------------- 5.0/11.6 MB 4.4 MB/s eta 0:00:02
   ------------------- -------------------- 5.8/11.6 MB 4.5 MB/s eta 0:00:02
   --------------------- ------------------ 6.3/11.6 MB 4.4 MB/s eta 0:00:02
   --------------------- ------------------ 6.3/11.6 MB 4.4 MB/s eta 0:00:02
   ----------------------- ---------------- 6.8/11.6 MB 3.5 MB/s eta 0:00:02
   ------------------------- -------------- 7.3/11.6 MB 3.5 MB/s eta 0:00:02
   --------------------------- ------------ 7.9/11.6 MB 3.3 MB/s eta 0:00:02
   ----------

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd

# 모델 불러오기
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']

# 이미지 전처리 함수
def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize_with_pad(img, 256, 256)
    input_img = tf.expand_dims(img, axis=0)
    input_img = tf.cast(input_img, dtype=tf.int32)
    return input_img

# keypoint 추출 함수
def detect_pose(image_path):
    input_img = load_and_preprocess_image(image_path)
    outputs = movenet(input_img)
    keypoints = outputs['output_0'].numpy()
    return keypoints[0, 0, :, :2]  # [17, (y, x)]

# 결과 저장 리스트
records = []

# Normal 폴더 경로
normal_root = 'G:/내 드라이브/Colab Notebooks/FallDetection/frames/Normal'

# normal_01 ~ normal_34 처리 (단, normal_28 제외)
for i in range(1, 35):
    if i == 28:
        continue  # normal_28 제외

    folder_name = f'normal_{i:02d}'
    folder_path = os.path.join(normal_root, folder_name)

    if not os.path.exists(folder_path):
        print(f"[경고] 폴더 없음: {folder_path}")
        continue

    for fname in sorted(os.listdir(folder_path)):
        if fname.endswith('.jpg') or fname.endswith('.png'):
            image_path = os.path.join(folder_path, fname)
            keypoints = detect_pose(image_path)

            for kp_idx, (y, x) in enumerate(keypoints):
                records.append({
                    'video_folder': folder_name,
                    'frame': fname,
                    'keypoint_index': kp_idx,
                    'x': float(x),
                    'y': float(y)
                })

# DataFrame으로 변환 및 저장
df = pd.DataFrame(records)
df.to_csv('keypoints_normal.csv', index=False)
print("✅ Normal keypoints 저장 완료: keypoints_normal.csv")

✅ Normal keypoints 저장 완료: keypoints_normal.csv


In [ ]:
import os
print(os.getcwd())

c:\Users\ASUS\AppData\Local\Programs\Microsoft VS Code


In [ ]:
df.to_csv('G:/내 드라이브/Colab Notebooks/FallDetection/keypoints_normal.csv', index=False)

In [ ]:
fall_root = 'G:/내 드라이브/Colab Notebooks/FallDetection/frames/Fall'
records = []

for i in range(1, 31):
    folder_name = f'fall_{i:02d}'
    folder_path = os.path.join(fall_root, folder_name)
    if not os.path.exists(folder_path):
        print(f"[❌ 경고] 없음: {folder_path}")
        continue

    for fname in sorted(os.listdir(folder_path)):
        if fname.endswith('.jpg') or fname.endswith('.png'):
            image_path = os.path.join(folder_path, fname)
            try:
                keypoints = detect_pose(image_path)
                for idx, (y, x) in enumerate(keypoints):
                    records.append({
                        'class': 'Fall',
                        'video_folder': folder_name,
                        'frame': fname,
                        'keypoint_index': idx,
                        'x': float(x),
                        'y': float(y)
                    })
            except Exception as e:
                print(f"[⚠️ 오류] {image_path} 실패: {e}")

df = pd.DataFrame(records)
df.to_csv('keypoints_fall.csv', index=False)
print("✅ keypoints_fall.csv 저장 완료") 

: 

: 

: 

: 

: 

In [ ]:

import os
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

# 모델 1회만 로드
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']

# 전처리 및 추론 함수
def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize_with_pad(img, 256, 256)
    input_img = tf.expand_dims(img, axis=0)
    input_img = tf.cast(input_img, dtype=tf.int32)
    return input_img

def detect_pose(image_path):
    input_img = load_and_preprocess_image(image_path)
    outputs = movenet(input_img)
    keypoints = outputs['output_0'].numpy()
    return keypoints[0, 0, :, :2]

# ✅ 기존 파일 불러오기 (있으면 이어서, 없으면 새로)
existing_csv = 'keypoints_fall.csv'
processed = set()

if os.path.exists(existing_csv):
    df_existing = pd.read_csv(existing_csv)
    processed = set(zip(df_existing['video_folder'], df_existing['frame']))
    print(f"🔁 이미 처리된 개수: {len(processed)//17}개 프레임")

# 경로 설정 (네 경로로 수정)
fall_root = 'G:/내 드라이브/Colab Notebooks/FallDetection/frames/Fall'
new_records = []

# 이어서 실행
for i in range(1, 31):
    folder_name = f'fall_{i:02d}'
    folder_path = os.path.join(fall_root, folder_name)
    if not os.path.exists(folder_path):
        print(f"❌ 폴더 없음: {folder_path}")
        continue

    for fname in sorted(os.listdir(folder_path)):
        if not (fname.endswith('.jpg') or fname.endswith('.png')):
            continue

        if (folder_name, fname) in processed:
            continue  # 이미 처리된 프레임이면 건너뜀

        image_path = os.path.join(folder_path, fname)
        try:
            keypoints = detect_pose(image_path)
            for idx, (y, x) in enumerate(keypoints):
                new_records.append({
                    'class': 'Fall',
                    'video_folder': folder_name,
                    'frame': fname,
                    'keypoint_index': idx,
                    'x': float(x),
                    'y': float(y)
                })
        except Exception as e:
            print(f"[⚠️ 오류] {image_path} 실패: {e}")

# 이어 붙이기 or 새로 저장
if os.path.exists(existing_csv):
    df_all = pd.concat([df_existing, pd.DataFrame(new_records)], ignore_index=True)
else:
    df_all = pd.DataFrame(new_records)

df_all.to_csv(existing_csv, index=False)
print("✅ keypoints_fall.csv 저장 완료 (중단 지점부터 이어서 실행됨)")